# RegEx in spaCy

spaCy has quick ways to implement RegEx in three pipes: 

- Matcher
- PhraseMatcher
- EntityRuler

Matcher and PhraseMatcher do not align the matched patterns as entities in the doc.ents. Thus we utilize EntityRuler to implement regular expressions.

#### 1. RegEx with EntityRuler

In [12]:
# import required libraries
import spacy

In [13]:
# Let us see an example

nlp = spacy.blank('en')

patterns = [{'label':'PHONE_NUMBER', 'pattern': [{'SHAPE':'ddd'},{'ORTH':'-'},{'SHAPE':'ddd'},{'ORTH':'-'},{'SHAPE':'dddd'}]}]

ruler = nlp.add_pipe('entity_ruler')

ruler.add_patterns(patterns)

In [14]:
text = 'Our phone number is 832-123-5555 and their phone number is 425-123-3829.'

doc = nlp(text)

In [15]:
print([(ent.text, ent.label_) for ent in doc.ents])

[('832-123-5555', 'PHONE_NUMBER'), ('425-123-3829', 'PHONE_NUMBER')]


Another Example

In [16]:
text = "Our phone number is 4251234567."

patterns = [{"label": "PHONE_NUMBERS", "pattern": [{"TEXT": {"REGEX": r"(\d){10}"}}]}]

ruler.add_patterns(patterns)

doc = nlp(text)
print([(ent.text, ent.label_) for ent in doc.ents])

[('4251234567', 'PHONE_NUMBERS')]
